## Combining Filters

像因子一样，过滤器可以结合使用。组合过滤器使用`&`(与）和`|`（或）运算。例如，假设我们想要筛选平均美元数量前10%并且最近收盘价在20美元以上的证券。首先，让我们使用`AverageDollarVolume`因子和`percentile_between`创建高美元成交过滤器：

注意：`percentile_between`是返回过滤器的因子方法。

接下来，让我们创建一个`latest_close`因子并为收盘价高于20美元的证券定义一个过滤器：

现在我们可以使用`&`运算符将我们的`high_dollar_volume`过滤器与我们的`above_20`过滤器结合起来：

is_tradeable = high_dollar_volume & above_20

对于`high_dollar_volume`和`above_20`均为`True`的证券，此过滤器将评估为`True`。否则，它将评估为`False`。类似的计算可以用`|`（或）操作符。

如果我们想在流水线中使用这个过滤器作为`screen`参数值，我们可以将`screen`设置为`is_tradeable`。

In [1]:
from zipline.pipeline.factors import AverageDollarVolume, SimpleMovingAverage
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.research import run_pipeline

In [2]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(
        window_length=30,
        inputs=[USEquityPricing.close, USEquityPricing.volume])
    high_dollar_volume = dollar_volume.percentile_between(90, 100)

    latest_close = USEquityPricing.close.latest
    above_20 = latest_close > 20

    is_tradeable = high_dollar_volume & above_20

    return Pipeline(
        columns={'percent_difference': percent_difference},
        screen=is_tradeable)

运行这条管道，2015年5月5日产出约130个证券。

In [3]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print('Number of securities that passed the filter: %d' % len(result))

Number of securities that passed the filter: 130


在下一课中，我们将看看掩蔽因素和过滤器。